In [1]:
%matplotlib widget
%load_ext autoreload
%autoreload 2
from IPython.display import clear_output
import os
import sys

# sys.path.insert(0, os.getenv('HOME')+'/pycode/MscThesis/')
# sys.path.insert(0,r'C:\Users\coren\Documents\PhD\Code\AMFtrack')

import pandas as pd
from amftrack.util.sys import (
    get_dates_datetime,
    get_dirname,
    temp_path,
    get_data_info,
    update_plate_info,
    get_current_folders,
    get_folders_by_plate_id,
)


import ast
from amftrack.plotutil import plot_t_tp1
from scipy import sparse
from datetime import datetime
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from amftrack.pipeline.functions.image_processing.extract_graph import (
    from_sparse_to_graph,
    generate_nx_graph,
    sparse_to_doc,
)
from skimage.feature import hessian_matrix_det

# from amftrack.pipeline.functions.image_processing.experiment_class_surf import Experiment
from amftrack.pipeline.launching.run_super import *
import dropbox
from amftrack.util.dbx import upload_folders, load_dbx, download, get_dropbox_folders
from subprocess import call
import logging

logging.basicConfig(stream=sys.stdout, level=logging.debug)

/gpfs/home2/cbisot/pycode/AMFtrack/amftrack/util/dbx.py:3: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
directory_targ = os.path.join(directory_scratch, "stitch_temp")

In [3]:
all_folders_drop = get_dropbox_folders("/DATA/PRINCE", True)

2022-06-09 19:47:16,285-[INFO]- dropbox:388 -> Refreshing access token.
2022-06-09 19:47:16,639-[INFO]- dropbox:477 -> Request to files/list_folder
2022-06-09 19:47:17,742-[INFO]- dropbox:477 -> Request to files/list_folder/continue
2022-06-09 19:47:18,777-[INFO]- dropbox:477 -> Request to files/list_folder/continue
2022-06-09 19:47:19,861-[INFO]- dropbox:477 -> Request to files/list_folder/continue
2022-06-09 19:47:20,828-[INFO]- dropbox:477 -> Request to files/list_folder/continue
2022-06-09 19:47:21,934-[INFO]- dropbox:477 -> Request to files/list_folder/continue
2022-06-09 19:47:23,086-[INFO]- dropbox:477 -> Request to files/list_folder/continue
2022-06-09 19:47:23,958-[INFO]- dropbox:477 -> Request to files/list_folder/continue
2022-06-09 19:47:25,278-[INFO]- dropbox:477 -> Request to files/list_folder/continue
2022-06-09 19:47:26,260-[INFO]- dropbox:477 -> Request to files/list_folder/continue
2022-06-09 19:47:27,205-[INFO]- dropbox:477 -> Request to files/list_folder/continue
20

In [7]:
folders_drop = all_folders_drop.loc[all_folders_drop["CrossDate"] >= "20220101"]

In [8]:
folders_drop

,folder,size,change_date,tot_path_drop,Plate,CrossDate,unique_id
0,20220419_2105_Plate27,NaN,2022-06-09 10:53:33,data/prince/6666668_20220330/6666666_20220328/...,6666666,20220328,6666666_20220328
1,20220408_2303_Plate01,NaN,2022-06-09 10:19:34,data/prince/6666668_20220330/6666666_20220328/...,6666666,20220328,6666666_20220328
2,20220414_1107_Plate01,NaN,2022-06-09 10:01:54,data/prince/6666668_20220330/6666666_20220328/...,6666666,20220328,6666666_20220328
3,20220424_2024_Plate27,NaN,2022-06-09 09:51:25,data/prince/6666668_20220330/6666666_20220328/...,6666666,20220328,6666666_20220328
4,20220409_2103_Plate01,NaN,2022-06-09 09:36:08,data/prince/6666668_20220330/6666666_20220328/...,6666666,20220328,6666666_20220328
...,...,...,...,...,...,...,...
15743,20220317_0021_Plate03,NaN,2022-05-18 15:15:47,data/prince/836_20220311/20220317_0021_plate03,836,20220311,836_20220311
15744,20220317_0015_Plate02,NaN,2022-05-18 15:15:47,data/prince/910_20220315/20220317_0015_plate02,910,20220315,910_20220315
15745,20220316_0014_Plate02,NaN,2022-05-18 15:15:45,data/prince/910_20220315/20220316_0014_plate02,910,20220315,910_20220315
15746,20220315_2220_Plate03,NaN,2022-05-18 15:15:43,data/prince/836_20220311/20220315_2220_plate03,836,20220311,836_20220311


In [9]:
run_parallel_transfer(
    "from_drop.py",
    [directory_targ],
    folders_drop,
    1,
    "30:00",
    "staging",
    cpus=1,
    node="staging",
)
clear_output(wait=False)

In [7]:
directory = directory_targ + "/"
directory = directory_project

In [8]:
update_plate_info(directory, local=True)
all_folders = get_current_folders(directory, local=True)

analysed:   0%|          | 0/243 [00:00<?, ?it/s]

In [9]:
folders = all_folders

In [10]:
folders

,total_path,Plate,PrincePos,root,strain,medium,split,Pbait,CrossDate,Temperature,...,/Analysis/transform.mat,/Analysis/transform_corrupt.mat,/Analysis/skeleton_realigned_compressed.mat,/Analysis/nx_graph_pruned.p,/Analysis/nx_graph_pruned_width.p,/Analysis/nx_graph_pruned_labeled.p,date,folder,unique_id,datetime
0,/projects/0/einf914/data/20220405_1343_Plate05,931,5,'Carrot','A5','001P100N100C','Y','-',20220402,25,...,False,False,False,False,False,False,"05.04.2022, 13:43:",20220405_1343_Plate05,931_20220402,2022-04-05 13:43:00
1,/projects/0/einf914/data/20220406_2233_Plate05,931,5,'Carrot','A5','001P100N100C','Y','-',20220402,25,...,False,False,False,False,False,False,"06.04.2022, 22:33:",20220406_2233_Plate05,931_20220402,2022-04-06 22:33:00
2,/projects/0/einf914/data/20220416_0936_Plate05,931,5,'Carrot','A5','001P100N100C','Y','-',20220402,25,...,False,False,False,False,False,False,"16.04.2022, 09:36:",20220416_0936_Plate05,931_20220402,2022-04-16 09:36:00
3,/projects/0/einf914/data/20220508_1248_Plate22,931,22,'Carrot','A5','001P100N100C','Y','-',20220402,25,...,False,False,False,False,False,False,"08.05.2022, 12:48:",20220508_1248_Plate22,931_20220402,2022-05-08 12:48:00
4,/projects/0/einf914/data/20220410_0316_Plate05,931,5,'Carrot','A5','001P100N100C','Y','-',20220402,25,...,False,False,False,False,False,False,"10.04.2022, 03:16:",20220410_0316_Plate05,931_20220402,2022-04-10 03:16:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,/projects/0/einf914/data/20220407_1320_Plate05,931,5,'Carrot','A5','001P100N100C','Y','-',20220402,25,...,False,False,False,False,False,False,"07.04.2022, 13:20:",20220407_1320_Plate05,931_20220402,2022-04-07 13:20:00
239,/projects/0/einf914/data/20220406_0348_Plate05,931,5,'Carrot','A5','001P100N100C','Y','-',20220402,25,...,False,False,False,False,False,False,"06.04.2022, 03:48:",20220406_0348_Plate05,931_20220402,2022-04-06 03:48:00
240,/projects/0/einf914/data/20220426_2113_Plate22,931,22,'Carrot','A5','001P100N100C','Y','-',20220402,25,...,False,False,False,False,False,False,"26.04.2022, 21:13:",20220426_2113_Plate22,931_20220402,2022-04-26 21:13:00
241,/projects/0/einf914/data/20220416_1739_Plate05,931,5,'Carrot','A5','001P100N100C','Y','-',20220402,25,...,False,False,False,False,False,False,"16.04.2022, 17:39:",20220416_1739_Plate05,931_20220402,2022-04-16 17:39:00


In [21]:
folders["CrossDate"].astype(str).unique()

array(['20220330', '20220411', '20220311', '20220402', '20220315',
       '20220309', '20220414', '20220325', '20220328', '20220222',
       '20220308', '20220317', '20220304', '20220419', '20220404',
       '20220321', '20220301', '20220406', '20220413', '20220422',
       '20220307', '20220323', '20220225', '20220101'], dtype=object)

In [18]:
folders.loc[folders["CrossDate"] == "nan"]

,total_path,Plate,PrincePos,root,strain,medium,split,Pbait,CrossDate,Temperature,...,/Analysis/transform.mat,/Analysis/transform_corrupt.mat,/Analysis/skeleton_realigned_compressed.mat,/Analysis/nx_graph_pruned.p,/Analysis/nx_graph_pruned_width.p,/Analysis/nx_graph_pruned_labeled.p,date,folder,unique_id,datetime
481,/scratch-shared/amftrack/stitch_temp/20220316_...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,...,False,False,False,False,False,False,"16.03.2022, 10:14:",20220316_1014_Plate02,nan_nan,2022-03-16 10:14:00


In [17]:
num_parallel = 50
time = "40:00"
run_parallel_stitch(directory, folders, num_parallel, time, cpus=128, node="fat")

sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 1.0 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 1.0 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


Submitted batch job 1098384
Submitted batch job 1098385
Submitted batch job 1098386


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 1.0 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 1.0 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


Submitted batch job 1098387


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 1.0 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 1.0 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


...

sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 1.0 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


Submitted batch job 1098405


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 1.0 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 1.0 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


Submitted batch job 1098406
Submitted batch job 1098407
Submitted batch job 1098408


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 1.0 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


In [22]:
args = []
num_parallel = 50
time = "20:00"
run_parallel_all_time(
    "make_video_stitched.py",
    args,
    folders,
    num_parallel,
    time,
    "make_video",
    cpus=128,
    node="thin",
    dependency=True,
    name_job="stitch.sh",
)

Submitted batch job 1098573
Submitted batch job 1098574


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 1.0 node. A full node consists of 128 CPU cores, 245760 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 1.0 node. A full node consists of 128 CPU cores, 245760 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


In [15]:
args = []
num_parallel = 50
time = "10:00"
run_parallel_all_time(
    "make_video_single.py",
    args,
    folders,
    num_parallel,
    time,
    "make_video",
    cpus=32,
    node="fat",
    dependency=True,
    name_job="stitch.sh",
)

Submitted batch job 1100454


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


In [12]:
folders

,total_path,Plate,PrincePos,root,strain,medium,split,Pbait,CrossDate,Temperature,...,/Analysis/transform.mat,/Analysis/transform_corrupt.mat,/Analysis/skeleton_realigned_compressed.mat,/Analysis/nx_graph_pruned.p,/Analysis/nx_graph_pruned_width.p,/Analysis/nx_graph_pruned_labeled.p,date,folder,unique_id,datetime
0,/projects/0/einf914/data/20220405_1343_Plate05,931,5,'Carrot','A5','001P100N100C','Y','-',20220402,25,...,False,False,False,False,False,False,"05.04.2022, 13:43:",20220405_1343_Plate05,931_20220402,2022-04-05 13:43:00
1,/projects/0/einf914/data/20220406_2233_Plate05,931,5,'Carrot','A5','001P100N100C','Y','-',20220402,25,...,False,False,False,False,False,False,"06.04.2022, 22:33:",20220406_2233_Plate05,931_20220402,2022-04-06 22:33:00
2,/projects/0/einf914/data/20220416_0936_Plate05,931,5,'Carrot','A5','001P100N100C','Y','-',20220402,25,...,False,False,False,False,False,False,"16.04.2022, 09:36:",20220416_0936_Plate05,931_20220402,2022-04-16 09:36:00
3,/projects/0/einf914/data/20220508_1248_Plate22,931,22,'Carrot','A5','001P100N100C','Y','-',20220402,25,...,False,False,False,False,False,False,"08.05.2022, 12:48:",20220508_1248_Plate22,931_20220402,2022-05-08 12:48:00
4,/projects/0/einf914/data/20220410_0316_Plate05,931,5,'Carrot','A5','001P100N100C','Y','-',20220402,25,...,False,False,False,False,False,False,"10.04.2022, 03:16:",20220410_0316_Plate05,931_20220402,2022-04-10 03:16:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,/projects/0/einf914/data/20220407_1320_Plate05,931,5,'Carrot','A5','001P100N100C','Y','-',20220402,25,...,False,False,False,False,False,False,"07.04.2022, 13:20:",20220407_1320_Plate05,931_20220402,2022-04-07 13:20:00
239,/projects/0/einf914/data/20220406_0348_Plate05,931,5,'Carrot','A5','001P100N100C','Y','-',20220402,25,...,False,False,False,False,False,False,"06.04.2022, 03:48:",20220406_0348_Plate05,931_20220402,2022-04-06 03:48:00
240,/projects/0/einf914/data/20220426_2113_Plate22,931,22,'Carrot','A5','001P100N100C','Y','-',20220402,25,...,False,False,False,False,False,False,"26.04.2022, 21:13:",20220426_2113_Plate22,931_20220402,2022-04-26 21:13:00
241,/projects/0/einf914/data/20220416_1739_Plate05,931,5,'Carrot','A5','001P100N100C','Y','-',20220402,25,...,False,False,False,False,False,False,"16.04.2022, 17:39:",20220416_1739_Plate05,931_20220402,2022-04-16 17:39:00


In [6]:
dir_drop = "DATA/PRINCE"
delete = True
run_parallel_transfer(
    "toward_drop.py",
    [dir_drop, delete],
    folders,
    1,
    "30:00",
    "staging",
    cpus=1,
    node="staging",
)
clear_output(wait=False)